In [1]:
from urllib.request import urlopen           # to download a webpage(html)
from bs4 import BeautifulSoup as bs          # to parse a webpage(html) and remove all crud present
import re

In [2]:
#Download all the url. Each url have 7 days blog posts
def getAllDoxyDonkeyPosts(url,links):
    page = urlopen(url)
    soup = bs(page)
    for a in soup.findAll('a'):        # will find all the links in the home page
        try:
            url = a['href']
            title = a['title']
            if title == "Older Posts":
                print(title , url)
                links.append(url)
                getAllDoxyDonkeyPosts(url,links)
        except:
            title = ""
    return

In [3]:
blogUrl = "https://doxydonkey.blogspot.in"
links = []
getAllDoxyDonkeyPosts(blogUrl,links)

Older Posts https://doxydonkey.blogspot.com/search?updated-max=2017-05-23T19:53:00-07:00&max-results=7
Older Posts https://doxydonkey.blogspot.com/search?updated-max=2017-05-14T19:02:00-07:00&max-results=7&start=7&by-date=false
Older Posts https://doxydonkey.blogspot.com/search?updated-max=2017-05-02T19:43:00-07:00&max-results=7&start=14&by-date=false
Older Posts https://doxydonkey.blogspot.com/search?updated-max=2017-04-17T19:26:00-07:00&max-results=7&start=21&by-date=false
Older Posts https://doxydonkey.blogspot.com/search?updated-max=2017-04-10T18:56:00-07:00&max-results=7&start=28&by-date=false
Older Posts https://doxydonkey.blogspot.com/search?updated-max=2017-03-30T19:57:00-07:00&max-results=7&start=35&by-date=false
Older Posts https://doxydonkey.blogspot.com/search?updated-max=2017-03-20T19:47:00-07:00&max-results=7&start=42&by-date=false
Older Posts https://doxydonkey.blogspot.com/search?updated-max=2017-03-02T17:42:00-08:00&max-results=7&start=49&by-date=false
Older Posts http

Older Posts https://doxydonkey.blogspot.com/search?updated-max=2015-05-04T20:23:00-07:00&max-results=7&start=455&by-date=false
Older Posts https://doxydonkey.blogspot.com/search?updated-max=2015-04-23T20:19:00-07:00&max-results=7&start=462&by-date=false
Older Posts https://doxydonkey.blogspot.com/search?updated-max=2015-04-14T19:40:00-07:00&max-results=7&start=469&by-date=false
Older Posts https://doxydonkey.blogspot.com/search?updated-max=2015-04-05T20:22:00-07:00&max-results=7&start=476&by-date=false
Older Posts https://doxydonkey.blogspot.com/search?updated-max=2015-03-24T20:12:00-07:00&max-results=7&start=483&by-date=false
Older Posts https://doxydonkey.blogspot.com/search?updated-max=2015-03-15T20:41:00-07:00&max-results=7&start=490&by-date=false
Older Posts https://doxydonkey.blogspot.com/search?updated-max=2015-03-03T19:30:00-08:00&max-results=7&start=497&by-date=false
Older Posts https://doxydonkey.blogspot.com/search?updated-max=2015-02-22T19:55:00-08:00&max-results=7&start=50

In [5]:
#Each day blog is contained in a div. In each day(divs) we may have multiple blogs which is represented by bullets(li in html)
# https://stackoverflow.com/questions/49145561/python-type-error-while-encoding-and-replacing-with-space

def getDoxyDonkeyText(url):
    page = urlopen(url)
    soup = bs(page)
    myDivs = soup.findAll("div",{"class":'post-body'})
    posts = []
    for div in myDivs:
         posts += map(lambda p: p.text.encode('ascii', errors='replace').decode().replace("?"," "), div.find_all('li'))
    return posts

In [6]:
doxyDonkeyPosts = []
for link in links:
    doxyDonkeyPosts+=getDoxyDonkeyText(link)

In [7]:
print(doxyDonkeyPosts)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_df=0.5,min_df=2,stop_words='english')  # tf-idf object
X = vectorizer.fit_transform(doxyDonkeyPosts)   #return 2-D matrix in which row is the number of articles and col is the number of distict words in all articles and each word in article is replaced by its tf-idf value 

In [13]:
from sklearn.cluster import KMeans

km = KMeans(n_clusters = 3, init = 'k-means++', max_iter = 100, n_init = 1,verbose = True) #kmeans object
km.fit(X)

Initialization complete
Iteration  0, inertia 5304.200
Iteration  1, inertia 2680.413
Iteration  2, inertia 2674.371
Iteration  3, inertia 2673.154
Iteration  4, inertia 2672.561
Iteration  5, inertia 2671.989
Iteration  6, inertia 2671.469
Iteration  7, inertia 2671.288
Iteration  8, inertia 2671.115
Iteration  9, inertia 2670.788
Iteration 10, inertia 2670.327
Iteration 11, inertia 2669.881
Iteration 12, inertia 2669.704
Iteration 13, inertia 2669.636
Iteration 14, inertia 2669.630
Converged at iteration 14: center shift 0.000000e+00 within tolerance 7.307995e-09


KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=100,
    n_clusters=3, n_init=1, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=True)

In [14]:
import numpy as np
np.unique(km.labels_,return_counts=True)   #Only to get the Details of the cluster. Number of cluster and number of articles within each cluster

(array([0, 1, 2]), array([ 252, 2375,  177], dtype=int64))

In [20]:
#To get the theme of each cluster
text = {}
print(km.labels_)
for i,cluster in enumerate(km.labels_):
    oneDocument = doxyDonkeyPosts[i]
    if cluster not in text.keys():
        text[cluster] = oneDocument
    else :
        text[cluster] += oneDocument

[1 1 2 ... 1 1 1]


In [17]:
import nltk
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.corpus import stopwords
from string import punctuation
from nltk.probability import FreqDist
from heapq import nlargest
from collections import defaultdict

In [19]:
customStopWords = set(stopwords.words('english')+list(punctuation) + ["million","billion","year","millions","billions","years"])

In [21]:
#Top(100) keywords in each cluster and their count
keywords = {}
counts = {}

for cluster in range(3):
    word_sent = word_tokenize(text[cluster].lower())
    word_sent = [word for word in word_sent if word not in customStopWords]
    freq = FreqDist(word_sent)
    keywords[cluster] = nlargest(100,freq,key=freq.get)
    counts[cluster]=freq

In [23]:
#Top(10) keywords unique to each cluster
unique_keys = {}

for cluster in range(3):
    otherClusters = list(set(range(3))-set([cluster]))
    keysOtherClusters = set(keywords[otherClusters[0]]).union(set(keywords[otherClusters[1]]))
    unique = set(keywords[cluster]) - keysOtherClusters
    unique_keys[cluster] = nlargest(10,unique,key=counts[cluster].get)

In [24]:
print(unique_keys)

{0: ['amazon', 'prime', 'delivery', 'items', 'cloud', 'amazon.com', 'retailer', 'echo', 'shipping', 'sellers'], 1: ['facebook', 'mobile', 'twitter', 'shares', 'internet', 'ads', 'software', 'since', 'stock', 'use'], 2: ['uber', 'lyft', 'drivers', 'didi', 'ride-hailing', 'cities', 'rides', 'car', 'private', 'driver']}


In [25]:
article = "Alibaba Group Holding Ltd.’s six-year-old excursion into mobile operating systems is faltering in China, casting doubt over software that bears billionaire-founder Jack Ma’s name and was once touted as key to countering Tencent Holdings Ltd. China’s largest e-commerce company debuted YunOS in 2011, a system that underpins search, shopping and browsing that its executives last year said could attain as much as 25 percent domestic market share by the end of 2016 -- surpassing Apple Inc.’s iOS. Six years on, YunOS’ slice of China software installations stands at just 2.2 percent while its share of 2016 shipments was 10 percent, researchers Canalys and Counterpoint estimate, respectively. Alibaba disputes those numbers. Alibaba managers have grown increasingly unhappy with its sluggish adoption and have begun an internal debate around the software’s future, a person familiar with the matter said. No conclusions have been reached, the person said, asking not to be named discussing a confidential matter. Yet the talks reflect the inability of a once-vaunted initiative to forestall Tencent’s dominance in the mobile arena, secured through the utility of WeChat -- a universal app that melds messaging, payments, media, shopping and on-demand services."

In [26]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier()
classifier.fit(X,km.labels_)                             #training

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [34]:
test = vectorizer.transform([article.encode('ascii',errors='ignore')])
test

<1x13220 sparse matrix of type '<class 'numpy.float64'>'
	with 90 stored elements in Compressed Sparse Row format>

In [35]:
classifier.predict(test)

array([1])